# 참조자료

-  https://smartshk.tistory.com/19


-  https://www.baeldung.com/kotlin/threads-coroutines

# 참조자료

-  https://rkdxowhd98.tistory.com/62?category=1134007

## 코루틴 모듈 올리기

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.1")

## 모듈 설치 확인하기

In [2]:
:classpath

Current classpath (14 paths):
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/lib-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/api-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.7.0-dev-1825.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.6.0.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx/kotlinx-coroutines-core/1.6.1/kotlinx-coroutines-core-1.6.1.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx

## 코루틴 컨텍스트 확인하기

In [3]:
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext

## launch() -> Job 반환

       : 결과가 없는 코루틴을 생성하는 빌더, 여기서 결과는 반환인스턴스가 아닌 결과값(Value)을 의미

         반환하는 Job인스턴스는 생성된 해당 코루틴을 제어하는데 사용

In [5]:
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext

//public fun CoroutineScope.launch(                            // 코루틴스코프 확장함수
//    context: CoroutineContext = EmptyCoroutineContext,       // 컨텍스트
//    start: CoroutineStart = CoroutineStart.DEFAULT,          // 스타트는 자동실행
//    block: suspend CoroutineScope.() -> Unit                 // 코루틴 로딩된 람다표현식
//): Job {                                                     // 반환객체
//    ...
//}

fun CoroutineScope.log(msg: String) {                          // 출력 확장함수 지정
   val name = coroutineContext[CoroutineName]?.name            // 코루틴컨텍스트 이름 검색
   println("[$name] $msg")                                     // 코루틴컨텍스트 이름 출력
}

fun main11() {
    GlobalScope.launch(CoroutineName("전역스코프")) {             // 전역스코프의 코루팀에 이름 정의 
        log("launch 코루틴 빌더 시작 ")
        var job = launch {
            delay(500)                                         // 0.5초 대기
            log("코루틴 처리 1")
        }
        
        job.join()                                             // 전역코루틴의 자식코루틴 종료까지 대기
        log("launch 코루틴 빌더 정지 ")                            // 최종 종료
    }
    Thread.sleep(1000)                                         // 메인스레드 잠시 대기
    println(" 메인스레드 처리 2")                                  // 메인출력 
}
main11()

[전역스코프] launch 코루틴 빌더 시작 
[전역스코프] 코루틴 처리 1
[전역스코프] launch 코루틴 빌더 정지 
 메인스레드 처리 2


## 다양한 코루틴 스코프 처리하기

In [16]:
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext

fun CoroutineScope.log(msg: String) {                      // 출력 확장함수 지정
   val name = coroutineContext[CoroutineName]?.name        // 코루틴컨텍스트 이름 검색
   println("[$name] $msg")                                 // 코루틴컨텍스트 이름 출력
}

fun main() = 
    runBlocking(CoroutineName("런블러킹")) {                 // 런블러킹 빌더에 코루틴이름 지정 
    val request = GlobalScope                              // 전역스코프 지정
                  .launch(CoroutineName("전역스코프"))  {     // 코푸틴에 전역스코프 이름 지정 
        launch(CoroutineName("새로운스코프") + Job()) {       //  자식 코루틴에 이름과 잡을 지정
            log("job1: 즉시실행 ->" 
                    + Thread.currentThread().name)         // 현재 스레드 출력 
            delay(1000)                                    // 1초 중단 
            log("job1: 다른 코루틴 처리후 실행-> "
                   + Thread.currentThread().name)          // 재시작후 출력 
        }                                                   
        launch {                                           // 전역스코프내의 코루틴 빌더 
            delay(100)                                     // 일시 중단 
            log("job2: 다른 코루틴 처리후 실행-> "
                    + Thread.currentThread().name)
            delay(1000)                                    // 일시 중단
            log("job2: 전체 코루틴 중단으로 미실행-> "
                    + Thread.currentThread().name)
        }
    }
    delay(5000)                                             // 일시중단 
    request.cancel()                                       // 글러벌스코프 전체 종료
    delay(1000)                                            // 종료 후 결과 처리
    log("main: 모든 고루틴 처리 후에 실행-> " 
                    + Thread.currentThread().name)
}
main()

[새로운스코프] job1: 즉시실행 ->DefaultDispatcher-worker-3
[전역스코프] job2: 다른 코루틴 처리후 실행-> DefaultDispatcher-worker-2
[새로운스코프] job1: 다른 코루틴 처리후 실행-> DefaultDispatcher-worker-2
[전역스코프] job2: 전체 코루틴 중단으로 미실행-> DefaultDispatcher-worker-2
[런블러킹] main: 모든 고루틴 처리 후에 실행-> Thread-73


## async() -> Deferred<T> 반환

       : launch와 다르게 결과를 가지는 코루틴을 생성하는 빌더,

         반환하는 Deferred<T>로 코루틴을 제어 / 결과를 받을 수 있는데

In [4]:
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext

// fun <T> CoroutineScope.async(                                  // 코루틴 스코프의 확장함수
//    context: CoroutineContext = EmptyCoroutineContext,          // 컨텍스트
//    start: CoroutineStart = CoroutineStart.DEFAULT,             // 스타트는 자동실행으로 지정 
//    block: suspend CoroutineScope.() -> T                       // 일시중단람다표현식 로딩
// ): Deferred<T>                                                 // 반환객체


fun CoroutineScope.log(msg: String) {
   val name = coroutineContext[CoroutineName]?.name
   println("[$name] $msg")
}

val syn = GlobalScope.async(CoroutineName("전역스코프")){             // 전역스코프에 코루틴 빌더 
    log("현재 코루틴 1 : " + Thread.currentThread().name)
    async(Dispatchers.Default) {                                   // 비동기 코뤁틴 빌더 
        delay(100)                                                 // 일시 정지 
        log("현재 코루틴 2 : " + Thread.currentThread().name)
    }
    
    delay(100)                                                     // 일지정지
    log("코루틴 종료 " + Thread.currentThread().name)                 // 부모를 종료처리 
}

Thread.sleep(2000)

[전역스코프] 현재 코루틴 1 : DefaultDispatcher-worker-2
[전역스코프] 현재 코루틴 2 : DefaultDispatcher-worker-2
[전역스코프] 코루틴 종료 DefaultDispatcher-worker-1


## 일시중단함수와 같이 처리하기

In [8]:
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext

fun CoroutineScope.log(msg: String) {                     // 확장함수
   val name = coroutineContext[CoroutineName]?.name       // 코루틴 이름
   println("[$name] $msg")
}

fun main11() = runBlocking(CoroutineName("런블러킹")){      // 코루틴 환경 
    val one = async { 
        doSomethingOne() 
    }                                                     // 동시처리
    val two = async { 
        doSomethingTwo() 
    }                                                     // 동시처리
    println(one.javaClass)
    log("Result : ${one.await() + two.await()}")          // 출력 
}

suspend fun doSomethingOne(): Int {
    delay(1000L)                                           // 일시중단
    return 10                                              // 결과 리턴
}
suspend fun doSomethingTwo(): Int {
    delay(1000L)                                           // 일시중단
    return 20                                              // 결과 리턴
}

main11()

class kotlinx.coroutines.DeferredCoroutine
[런블러킹] Result : 30


## 다양한 스코프에서 처리하기

In [6]:
fun CoroutineScope.log(msg: String) {                     // 확장함수
   val name = coroutineContext[CoroutineName]?.name       // 코루틴 이름
   println("[$name] $msg")
}

fun main11() = 
    runBlocking(CoroutineName("런블러킹")){                  // 코루틴 환경 
    val deferred = async {                                 // async로 코루틴 생성
        delay(1000)                                        // 1초 대기
        log("async Start")
        "async result"                                     // 해당 코루틴블럭(Deferred)의 await() 호출 시 반환 값
    }
    log("Test")
    val job = GlobalScope
              .launch(CoroutineName("전역스코프")) {          // launch로 코루틴 생성
        try {
            delay(200)                                     // 0.2초 대기
            log("launch Start") 
            delay(Long.MAX_VALUE)                          // 최대 시간 정지
        } finally {
            log("launch Cancelled")                        // 해당 job의 cancel() 취소 시 호출되는 finally
        }
    }
    delay(500)                                             // 0.5초 대기
    job.cancelAndJoin()                                    // job 취소, 대기
    delay(500)                                             // 0.5초 대기
    log(deferred.await())                                  // deferred의 결과 값 얻기
}
main11()

[런블러킹] Test
[전역스코프] launch Start
[전역스코프] launch Cancelled
[런블러킹] async Start
[런블러킹] async result


## 지연실행

In [10]:
fun main1() = runBlocking<Unit> {
    val deferred = async(start = CoroutineStart.LAZY) { // LAZY로 start 설정
        println("async Start")
        "async Result"
    }
    println("Test")

    val job = GlobalScope
              .launch(start = CoroutineStart.LAZY) {    // LAZY로 start 설정
        println("launch Start")
    }

    job.start()                                         // launch는 start() 또는 join()으로 해당 코루틴을 실행
    // job.join()                                       // start와 같이 실행시키고 join은 대기까지

    // deferred.start()                                 // async는 start() 또는 await()으로 해당 코루틴을 실행
    deferred.await()                                    // await도 실행시키고 대기까지
}

main1()

Test
async Start
launch Start


## 일시중단 함수와 중단 처리

In [11]:
fun main12() = runBlocking {                            // 런블러킹으로 코루틴 
    val one = async {                                   //  비동기 코루틴에서 일시중단함수 호출
        doSomethingOne()                                // 첫번째 일시중단함수 호출 
    }
    val two = async(start = CoroutineStart.LAZY) {      // 비동기 코루틴에서 지연 시작처리 
        doSomethingTwo()                                // 두번째 일시중단함수 호출
    }

    println("결과값 : ${one.await() + two.await()}")     // 지연처리한 것은 await 만나야 시작 

}

suspend fun doSomethingOne(): Int {                     // 일지중단함수 선언 
    delay(1000L)                                        // 일시중단
    println(" 10 반환 처리")
    return 10                                           // 결과 리턴
}

suspend fun doSomethingTwo(): Int {                     // 일시중단함수 선언 
    delay(1000L)                                        // 일시중단
    println(" 20 반환 처리 ")
    return 20                                           // 결과 리턴
}
main12()

 10 반환 처리
 20 반환 처리 
결과값 : 30


## withContext() - T 반환

       : async처럼 결과값을 반환하는 빌더로 async와 유사하지만 다른점이 존재한다

         async는 반환하는 Deferred<T> 객체로 결과값을 원하는 시점에 await()함수를 통해 결과값을 얻지만,

         withContext()는 Deferred<T>객체로 반환하지 않고, 결과(T)를 그 자리에서 반납한다 

 ( async의 Deferred<T>는 지연객체라고 생각하면 좋다. 내가 원하는 시점에 await()으로 결과를 받기에 - 지연 )

In [12]:
// suspend fun <T> withContext(                        // 일시중단함수 
// context: CoroutineContext,                          // 코루틴 컨텍스트
// block: suspend CoroutineScope.() -> T): T           // 일시중단 코루틴 람다표현식

fun main() = runBlocking {                             // 현재 스레드 코뤁틴 처리                       
    withContext(Dispatchers.Default) {                 // 컨택슽틑 변경 
        println("위드컨텍스트 처리 : " 
                + Thread.currentThread().name)
        delay(100)
    }
    
    launch {                                           // 기존 컨텍스트 사용
        println("런치 처리 : " 
                + Thread.currentThread().name)
        delay(100)
    }
    
    delay(2000)
}

main()

위드컨텍스트 처리 : DefaultDispatcher-worker-2
런치 처리 : Thread-49


## 이름 부여 처리하기ㅡ

In [18]:
import kotlinx.coroutines.*
import kotlin.coroutines.coroutineContext

suspend fun printName(msg : String) {                       // 일시중단함수에 코뤁틴컨텍스트 이름 출력
    val name = coroutineContext[CoroutineName]?.name
   println("[${name}]" + msg )
}

suspend fun main_() = 
    withContext(Dispatchers.Default 
                + CoroutineName("Outer" )) {  
                                                            // 특정 코루틴이름을 부여한 일시중단함수
       printName("시작" + Thread.currentThread().name)       // Outer
       launch(CoroutineName("Inner")) {                     // 자식코루틴에도 이름 붕여 
           printName("종료" + Thread.currentThread().name)   // Inner
       }
      delay(10)                                              // 지연
      printName("메인종료")                                    // Outer
}

fun main() = runBlocking {
      main_()
}

main()

[Outer]시작DefaultDispatcher-worker-2
[Inner]종료DefaultDispatcher-worker-1
[Outer]메인종료


## 반환값 처리

withContext + NonCancellable을 사용하면 코루틴

In [14]:
fun main2() = 
    runBlocking {                                         // 코루틴블럭 스코프 생성
        var jobw =  Any()
        val job = launch {                                // 자식 코루틴 새성 
            try {
                repeat(1000) { i ->                       // 1000개의 코루틴 생성 
                        println("코루틴 반복처리 $i ...")
                        delay(500L)
                }
            } finally {                                   // 종료하면 내부적으로 예외발생
                jobw = withContext(NonCancellable) {      // 무조건 처리가 가능한 코루틴 처리 
                    delay(1000)
                    println("main : 현재 상태 전부처리!")
                    100                                   // 반환값 
                }
            }
        }
        println("main : 메인대기중!")
        delay(1300L)                                       // 메인 일시중단 
        job.cancelAndJoin()                                // 코루틴 완전 중단 
        println("main : 메인 종료.")
        println(jobw.javaClass)
        if (jobw is Int)
            println(" ${jobw}")
}
    
main2()

main : 메인대기중!
코루틴 반복처리 0 ...
코루틴 반복처리 1 ...
코루틴 반복처리 2 ...
main : 현재 상태 전부처리!
main : 메인 종료.
class java.lang.Integer
 100


## withTimeout 

In [15]:
//suspend fun <T> withTimeout(                                // 일시중단함수 
//timeMillis: Long,                                           // 대기후 종료할 시간
//block: suspend CoroutineScope.() -> T): T                   // 코루틴 처리 람다표현식


fun main() = runBlocking {                                    // 런블러킹 코루틴
    try {
        withTimeout(1300L) {                                  // 타임아웃 처리
            repeat(1000) { i -> println("코루틴 처리 $i ...")    // 1000개의 자식 코루틴  
                delay(500L)                                   // 일시정지
            } 
        } 
    } catch(e: TimeoutCancellationException) {                // 예외 처리 
        println("타임아웃으로 종료 처리!!!")                        // 출력
    } finally {                                               // 종료하면 내부적으로 예외발생
        val jobw = withContext(NonCancellable) {              // 무조건 처리가 가능한 코루틴 처리 
                    delay(1000)
                    println("main : 현재 상태 전부처리!")
                    100                                       // 반환값 
                }
        println(jobw)
    }

}

main()

코루틴 처리 0 ...
코루틴 처리 1 ...
코루틴 처리 2 ...
타임아웃으로 종료 처리!!!
main : 현재 상태 전부처리!
100
